**GEO6361 (semaine 7) : Application de Pandas + Introduction à GeoPandas**

Cette semaine, nous allons utiliser la bibliothèque Pandas sur un jeu de données, et nous allons découvrir l'extension spatiale de Pandas (GeoPandas).

Nous verrons comment :
1. Manipuler des DataFrames
2. Visualiser leur contenu (surtout des distributions)
3. Faire des jointures spatiales
4. Produire des cartes.

## **1. Application de Pandas**

### **1.1 Importons Pandas et les bilbiothèques utiles**

In [ ]:
# Tout d'abord, on importe Pandas car c'est un module externe (mais préinstallé dans Google Colab)
import pandas as pd

# On importe également NumPy
import numpy as np

# Et le module maths dont on aura besoin dans les démonstrations
import math

### **1.2 Lisons notre fichier CSV contenant les données et créons une DataFrame**

In [ ]:
df = pd.read_csv('/content/data_immo_quebec_2023.csv') # Les données sont aimablement fournies par Thibault Lecorre (UdeM)

**À quoi ressemblent les donnnées ?**

Affichons les premières lignes (pour se faire une idée)

In [ ]:
df.head()

**Combien d'enregistrements le jeu de données possède-t-il ?**

In [ ]:
print(f"Le jeu de données possède {len(df)} lignes.")

**On retire tous les enregistrements dont le prix est inférieur à 10,000\$**

In [ ]:
# Filtrer des données
print(len(df))
df = df[(df['Prix'] > 10000)]
print(len(df))

### **1.3 Gestion des données manquantes ?**

In [ ]:
df.isnull()

In [ ]:
df.isnull().sum()

**Certaines variables possédant des valeurs manquantes ne nous sont pas vraiment nécessaires:**

In [ ]:
df = df.drop(['Num_Street', 'Street', 'BathRoom', 'ID', 'Unnamed: 0'], axis='columns')

**On décide (c'est une décision contextuelle) de supprimer les lignes dont certaines données sont manquantes**

In [ ]:
ndf = df.dropna()
print(f"Le jeu de données possède désormais {len(ndf)} lignes.")

**Revérifions la quantité de données nulles**

In [ ]:
ndf.isnull().sum()

**Trouver la liste des villes disponibles dans les données** (City)

In [ ]:
villes = ndf['City'].unique()
print(f"Le jeu de données contient {len(villes)} villes")

**Et à Montréal ?**

In [ ]:
c = 1
for city in ndf['City'].unique():
    if 'Montréal' in city:
        print(c, city)
        c += 1
print()

In [ ]:
# On peut aussi faire comme ceci:
ndf2 = ndf[ ndf['City'].str.contains('Montréal')]
ndf[ ndf['City'].str.contains('Montréal')]['City'].unique()

### **1.4 Quelques figures**

#### **Pandas peut produire des figures d'assez bonne qualité.**

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html

**Histogramme des prix :**

In [ ]:
taille_classe = 1e5 # dessiner l'histogramme par tranches de 100,000$
nbins = math.ceil((ndf['Prix'].max() - ndf['Prix'].min()) / taille_classe) # calcul du nombre de classes en fonction de leur taille
ndf['Prix'].plot.hist(bins=nbins, xlim=(0,2e6))

**Pointes de tarte pour les variables catégorielles :**

Avec la variable "Category" :

In [ ]:
ndf['Category'].value_counts()

In [ ]:
ndf['Category'].value_counts().plot.pie(figsize=(6, 6));

Avec la variable "BedRoom" :

In [ ]:
ndf['BedRoom'].value_counts().plot.pie(figsize=(6, 6));

####**On peut également utiliser un autre module : Seaborn.** Voir : https://seaborn.pydata.org

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set_style("whitegrid")
fig = sns.displot(ndf2, # Voir la documentation : https://seaborn.pydata.org/generated/seaborn.displot.html
                  x="Prix", # la colonne à visualiser
                  hue="Category", # la catégorie à partir de laquelle nous voulons séparer l'histogramme
                  height=5, # la hauteur de la figure
                  aspect=1.5, # le rapport hauteur/largeur
                  fill=True, # remplir l'histogramme d'une couleur (avec transparence)
                  kind="hist", # type de représentation (on peut aussi passer "kde" pour calculer la densité)
                  binwidth=50000) # largeur des classes
fig.set(xlim=(0, 2e6)) # pour limiter l'emprise des données
fig.set(title='Distribution des prix des biens immobiliers', xlabel='Prix', ylabel='Fréquence') # Pour modifier les titres et noms des axes
fig._legend.set_title('Type de bien') # Titre de la légende
fig.savefig("/content/figure_1_histogramme.png", dpi=500) # Pour exporter la figure

In [ ]:
sns.set_style("whitegrid")
fig = sns.displot(ndf,
                  x="Prix",
                  hue="Category",
                  height=5,
                  aspect=1.5,
                  fill=False,
                  kind="kde",
                  common_norm=False,)
fig.set(xlim=(0, 2e6)) # pour limiter l'emprise des données
fig.set(title='Distribution des prix des biens immobiliers par type', xlabel='Prix', ylabel='Densité') # Pour modifier les titres et noms des axes
fig._legend.set_title('Type de bien') # Titre de la légende
fig.savefig("/content/figure_1_densité.png", dpi=500) # Pour exporter la figure

**Afficher des statistiques descriptives par quartier :**

In [ ]:
ndf2.groupby('City')['Prix'].describe().transpose()

In [ ]:
sns.set_style("whitegrid")
fig = sns.displot(ndf2,
                  x="Prix",
                  hue="Category",
                  common_norm=False,
                  height=5,
                  aspect=2,
                  fill=True,
                  linewidth = 1,
                  kind="kde")
fig.set(xlim=(0, 2e6)) # pour limiter l'emprise des données
fig.set(title="Distribution des prix des biens immobiliers sur l'île de Montréal", xlabel='Prix', ylabel='Densité') # Pour modifier les titres et noms des axes
fig._legend.set_title('Arrondissement') # Titre de la légende
fig.savefig("/content/figure_2_densité.png", dpi=500) # Pour exporter la figure

In [ ]:
# Produisons un graphique de corrélations sur un sous-tableau :
ndf_pairs = ndf2[['Category', 'Prix', 'BedRoom']]

In [ ]:
ndf_pairs.head()

In [ ]:
# Explorer des relations :
sns.pairplot(ndf_pairs, hue="Category", height=4, aspect=1)

## **2. GeoPandas (Pandas spatial)**

**Attention** : avant d'utiliser GeoPandas, nous devons installer des bibliothèques externes. Pour cela, décommentez les lignes de la cellule suivante et exécutez-la (ça peut prendre du temps !). Recommentez ensuite ces lignes pour ne pas les ré-exécuter par inadvertance.

### **2.1 Installons les modules requis pour cette section, et importons-les**

In [ ]:
# %%capture fait en sorte que les multiples sorties de la cellule ne soient pas affichées dans la console
%%capture

#!pip install geopandas
!pip install mapclassify
!pip install contextily

In [ ]:
# On importe GeoPandas et on lui attribue l'alias gpd
import geopandas as gpd

### **2.2 Produisons des couches spatiales**

#### **Convertir notre Dataframe en GeoDataFrame**

In [ ]:
# on peut créer une GeoDataFrame à partir d'une DataFrame possédant des information spatiales
gdf = gpd.GeoDataFrame(ndf, geometry=gpd.points_from_xy(ndf['lon'], ndf['lat']))
gdf.crs = "EPSG:4326" # Pour attribuer un système de projection
gdf = gdf.to_crs("epsg:32188") # Pour reprojeter (MTM Zone 8)

In [ ]:
# On peut produire des cartes simples très rapidement avec GeoPandas
gdf.plot(markersize=5,
         figsize=(10, 10))

#### **Créons une couche GPD à partir d'un GeoJSON externe**

In [ ]:
quartiers = gpd.read_file('/content/quartiers_mtl.geojson')
print(quartiers.crs)

# Spécifier le système de projection du fichier (c'est du WGS84, donc EPSG4326) car GeoJson ne contient pas l'information de CRS
quartiers.crs = "EPSG:4326"

# Le convertir en ESPG32188
quartiers = quartiers.to_crs("epsg:32188")

# Afficher les données
quartiers.plot(markersize=5, figsize=(12, 12),
         alpha=0.5,
         #color='blue',
         edgecolor='black')

### **2.3. Faire une jointure spatiale pour calculer les prix par quartiers**


**On possède une gdf des biens immobiliers au Québec, et une gdf des polygones des quartiers de l'île de Montréal. Nous voulons combiner ces deux jeux de données**.

Pour rappel, dans le cas des jeux de données aspatiales, nous pouvons combiner les informations de deux jeux de données en faisant correspondre les valeurs d'une colonne. Ainsi, si deux enregistrements partagent la même valeur pour cette colonne en particulier, alors les informations seront concaténées.

Avec les jeux de données spatiales, nous pouvons aussi utiliser les relations spatiales pour combiner deux jeux de données. Ainsi, si une relation spatiale est identifiée entre deux enregistrements, alors les informations seront concaténées.

Ici, nous voudrions concaténer les informations sur les polygones des quartiers aux enregistrements sur les biens immobiliers. C'est-à-dire, si un logement est géographiquement localisé dans un quartier, alors les informations du quartier seront ajoutées. À la fin de la jointure, la gdf des biens immobiliers sera enrichie des informations spatiales (le polygone) des quartiers.


In [ ]:
# Associer chaque annonce au polygone du quartier auquel elle appartient :
prix_quartier = gpd.sjoin(quartiers, gdf, how="inner", predicate='contains') # https://geopandas.org/en/stable/gallery/spatial_joins.html
#quartiers.sindex.valid_query_predicates
#inner = intersection entre les deux jeux de données
#contains = relation spatiale
prix_quartier.tail()

**Nous pouvons maintenant effectuer une opération de dissolution (aggregation) sur nos enregistrements**

Tous les enregistrement possèdant le même nom de quartier seront aggrégés et leurs colonnes moyennées.

In [ ]:
prix_quartier2 = prix_quartier[['Prix', 'nom','geometry',]]
prix_quartier_dissolved = prix_quartier2.dissolve(by='nom', aggfunc='mean') #numeric_only=True, or several categories for the by argument

### **2.4. Un peu de cartographie (simple)**

**Produisons une carte choroplèthe des prix moyens par quartier.**

In [ ]:
prix_quartier_dissolved.plot(column='Prix',
                             legend=True,
                             cmap='OrRd',
                             scheme='natural_breaks',
                             k=10,
                             figsize=(10, 10))

**Ajouter un fond de carte (personnalisé)**

In [ ]:
import contextily as cx

In [ ]:
prix_quartier_dissolved = prix_quartier_dissolved.to_crs('EPSG:3857')

In [ ]:
couche_1 = prix_quartier_dissolved.plot(column='Prix',
                                 legend=True,
                                 cmap='OrRd',
                                 scheme='natural_breaks',
                                 alpha=0.5,
                                 figsize=(10, 10))

cx.add_basemap(couche_1,
               zoom=12
               #source=cx.providers.CartoDB.PositronNoLabels # différents styles : https://geopandas.org/en/stable/gallery/plotting_basemap_background.html
               )
cx.add_basemap(couche_1, source=cx.providers.CartoDB.PositronOnlyLabels, zoom=11)


couche_1.set_axis_off()

plt.savefig('/content/prix_par_quartier.png', dpi=500)

### **2.4. Exporter des données spatiales**

In [ ]:
prix_quartier_dissolved.to_file("/content/prix_par_quartier.geojson", driver='GeoJSON')